## End-to-[not quite]-end application to predicting housing values

This workbook includes a nearly complete, end-to-end data analytics project. We'll stop short of what is typically done since we will only construct and validate a single model. Typically we'll construct many models, iterate through the training and testing phase multiple times, and then ultimately proceed with validating and operationalizing a final "best" model. Our major steps in this workbook are as follows:
+ Load the libraries and read in the data
+ Identify whether we should expect any difficulties due to missing data
+ Split the entire dataset into *training*, *test*, and *safe* sets (we'll build our own function for this, but I'll point out an existing function that comes as part of the `sklearn` library)
+ Engage in some data visualization
+ Deal with NA (missing or null) values
+ Encode categorical variables so that they are able to be utilized by a mathematical model
+ Chain data cleaning, feature engineering, and model fitting together into a pipeline which can be automated
+ Evaluate our model's performance

Beware that this notebook includes quite a bit of material that we have not yet covered. We will be building up to something like this for the first half of our course together. You will not understand all of the code (and you are not expected to) -- focus on following the procedure. Identify where the steps I've outlined above are implemented, and note the flow of the procedure. An analytics project is not a linear venture -- there is lots of iterating and circling back. See if you can identify the spots where we are circling back to earlier stages -- why are we doing it? Again, focus not on the code, but on the procedure being followed -- you will become more familiar with the coding as we gain more experience throughout the semester.

#### Importing libraries and data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

housing = pd.read_csv("https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv")
housing.head()

#### Exploring the data

In [ ]:
housing.info()

From the output above we can see that the only column containing missing values is the `total_bedrooms` column. All other columns contain 20640 non-null entries, which matches the full length of the dataset. We know that we'll eventually need to fill these in. For this analytics application, let's assume that we are working for a real estate company which is predicting median home values in various neighborhoods. We'll start by making a random split into *training*, *test*, and *safe* sets.

#### Splitting the data into training and test sets

The `sklearn` library contains a function called `train_test_split()` which can be used to split a dataframe into training and test sets. We'll create our own function for two reasons -- firstly, it is useful to understand how some of these *built-in* functions are constructed, and secondly, we want a training set, test set, and also a safe set. While our function does not behave exactly like `train_test_split()`, it works quite similarly.

In [ ]:
#random sampling "by hand"
def split_train_test_safe(data, train_ratio, test_ratio, rand_seed):
    np.random.seed(rand_seed)
    shuffled_indices = np.random.permutation(len(data))
    train_set_size = int(len(data)*train_ratio)
    test_set_size = int(len(data) * test_ratio)
    train_indices = shuffled_indices[:train_set_size]
    test_indices = shuffled_indices[train_set_size:(train_set_size + test_set_size)]
    safe_indices = shuffled_indices[(train_set_size + test_set_size):]
    return data.iloc[train_indices], data.iloc[test_indices], data.iloc[safe_indices]

Now that our `split_train_test_safe()` function is defined, let's use it to split our data into the three subsets!

In [ ]:
train_set, test_set, safe_set = split_train_test_safe(housing, 0.75, 0.15, 370)

We can run a quick test to determine whether our function worked as expected. Below we print out the sizes (in terms of number of observations) of each of our subsets. The last line does a *lazy* test to determine whether the collection of the three subsets covers the entire dataset.

In [ ]:
print(len(train_set))
print(len(test_set))
print(len(safe_set))
print(len(train_set) + len(test_set) + len(safe_set) == len(housing))

Okay, so our custom function worked as expected. We now have three random subsets of data -- a *training set* which contains roughtly 75% of our observations, a *test set* containing about 15% of our observations, and a *safe set* which contains about 10% of our observations. We should note that our three subsets resulted from "random sampling" which means that the distributions of some of the variables in our subsets may (by chance) not be proportional to their prevalence in the overall dataset. If there are some variables which you want truly proportional representation of, then `sklearn` offers a function called `StratifiedShuffleSplit()` which allows us to define a collection of variables which we would like to stratify by, ensuring nearly proportional representation. This technique is beyond the scope of our course, but is typically useful in cases where we have categorical variables in which one or more classes may be relatively rare compared to others. This is a particular concern if the categorical variable is our response variable. 

## Exploratory Data Analysis

Any analysis starts with exploratory data analysis (commonly referred to as EDA). We'll begin here too. There are several features which are worth better understanding through summary statistics and data visualization. Let's start with a scatterplot of longitude and latitude -- can you identify the real estate market we are working with?

In [ ]:
train_set.plot(kind = "scatter", x = "longitude", y = "latitude")

Remember that our training data contained 15,480 observations. The plot above is informative -- you can likely tell what state we are working with, but it suffers from *overplotting*. The plot doesn't do a great job of showing us the *density* of the observations -- are there some locations with more (or fewer) observations? We can get a better view if we add transparency to the plot. Transparency is governed by the `alpha` parameter.

In [ ]:
train_set.plot(kind = "scatter", x = "longitude", y = "latitude", alpha = 0.1)

That's better! Can we add some more information to this plot without cluttering it too much? What if we size the points by population (in hundreds of residents) and color according to median home value? Do population centers have higher or lower median home values? Is there a *geo-spatial* explanation for property wealth?

In [ ]:
train_set.plot(kind = "scatter",
            x = "longitude",
            y = "latitude",
            alpha = 0.4,
            s = housing["population"]/100,
            label = "population",
            figsize = (10, 7),
            c = "median_house_value",
            cmap = plt.get_cmap("jet"),
            colorbar = True)
plt.legend()

#### Looking for correlations

Since we are interested in predicting `median_house_value` it is worth understanding which variables are most highly correlated to this response. Note, however, that correlations are only meaningful for numerical variables -- this means that the `ocean_proximity` variable will not be present in a correlation matrix -- don't forget that it exists!

In [ ]:
corr_matrix = train_set.corr()
corr_matrix["median_house_value"].sort_values(ascending = False)

Sometimes a visual is more desireable than a table of values. Let's use `seaborn`'s `pairplot()` method to construct a panel of scatterplots for `median_house_value` and the variables most strongly correlated with it.

In [ ]:
attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
sns.pairplot(train_set[attributes], kind = "scatter", plot_kws = {"alpha" : 0.2})

The plot seems to show a strong association between `median_income` and `median_house_value` -- this is verified by the correlation matrix we computed earlier. Let's isolate that plot.

In [ ]:
train_set.plot(kind = "scatter", x = "median_income", y = "median_house_value", alpha = 0.1)

You may notice a "weird" feature of this plot is the pretty dense horizontal collection of points at the \\$500,000 mark for median home value. This is because (for some reason), all neighborhoods with median home values exceeding \\$500,000 were just listed at \\$500,000 -- sometimes very questionable data collection and maintanence decisions are made, and unfortunately analysts need to deal with the repercussions.

#### Feature Engineering

Now that we've engaged in some exploratory analysis, it may be time to experiment with feature engineering. The correlations between `total_rooms` and `total_bedrooms` with `median_house_value` were disappointing -- this is because `total_rooms` and `total_bedrooms` is a metric covering all of the homes in the corresponding neighborhood (again, weird data collection practices). We would likely do better if we could get a metric for average rooms per household, average bedrooms per household, and maybe average number of residents per household. We'll engineer those features below. There are other features which could be useful too -- if you can think of any, try adding them as well.

Notice that we are now going back to the original `housing` dataset to construct these features. We do this so that we do not need to replicate the reconstruction on each of the *training set*, *test set*, and *safe set*. Just because we add these features to the original dataframe, they are not automatically added to our subsets, so we will need to re-run our `split_train_test_safe()` function -- since we set a seed for our random number generator, we are able to ensure that exactly the same observations end up in each subset after we re-split.

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_per_household"] = housing["total_bedrooms"] / housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["population_per_household"] = housing["population"] / housing["households"]

train_set, test_set, safe_set = split_train_test_safe(housing, 0.75, 0.15, 370)

In [ ]:
corr_matrix = train_set.corr()
corr_matrix["median_house_value"].sort_values(ascending = False)

Notice now that `rooms_per_household` has a stronger positive correlation with `median_house_value` than `total_rooms` did, and `bedrooms_per_room` has a low-moderate negative correlation with `median_house_value` while `total_bedrooms` had no correlation with `median_house_value`. Including these new features results in a big improvement! 

#### Cleaning the Data

We still haven't dealt with those pesky missing values, and we've got the `ocean_proximity` column which we haven't even considered yet. Its been left out of most of our analysis because it is categorical, so metrics like correlation don't make sense. We do, however, intuitively understand that neighborhoods which abut the ocean typically have higher property values than those which are further away from the ocean. It makes sense to expect that `ocean_proximity` will be a valuable piece of information to use when predicting `median_house_value`.

Let's start by taking care of the missing values in the `total_bedrooms` column. Remember that we computed our useful `bedrooms_per_room` feature from this column, so we'll need to recalculate that and also re-split into the *training*, *test*, and *safe* sets afterwards. There are some intricacies through this section, so take it slowly.

To deal with missing values, we have the following options:
+ Eliminate rows with missing data
    + `housing.dropna(subset = ["total_bedrooms"])`
+ Eliminate column containing missing values
    + `housing.drop("total_bedrooms", axis = 1)`
+ Impute with mean, median, etc. Be sure to use only the training data to compute these values, and then fill all missing values with the computed metric from the *training* set.
    + `median = train_set["total_bedrooms"].median()`
      
      `housing["total_bedrooms"].fillna(median, inplace = True)`
+ We can also use the `sklearn` `SimpleImputer` to fill all missing values according to a strategy. This is convenient since it can deal with many columns all at once, but we must treat numeric and categorical columns separately and then re-join them together when using `SimpleImputer`.
    + `from sklearn.impute import SimpleImputer`
    
      `imputer = SimpleImputer(strategy = "median")`
      
      Remove any non-numerical columns since median is irrelevant for non-numerics. We do this for both the training set and the original dataset since we will *fit* the imputer on the *training* data and then use it to *transform* the original dataset. The dataset which the "transformer" is applied to must have the same features as the dataset it was "fit" on.
      
      `train_set_num = train_set.drop("ocean_proximity", axis = 1)`
      
      `housing_num = housing.drop("ocean_proximity", axis = 1)`
      
      Fit the imputer to the training data.
      
      `imputer.fit(train_set_num)`
      
      You can see that `imputer.statistics_` matches `train_set_num.median().values` 
      
      Now we fill the missing values in the original dataset with the imputed values.
      
      `X = imputer.transform(housing_num)`
      
      The result is converted to a `numpy` array which needs to be put back into a `pandas` dataFrame.
      
      `housing_num_tr = pd.DataFrame(X, columns = housing_num.columns, index = housing_num.index)`

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy = "median")
train_set_num = train_set.drop("ocean_proximity", axis = 1)
housing_num = housing.drop("ocean_proximity", axis = 1)
imputer.fit(train_set_num)
X = imputer.transform(housing_num)

housing_num_tr = pd.DataFrame(X, columns = housing_num.columns, index = housing_num.index)
housing_num_tr.info()

Okay, now we've filled in all of the missing values in the `total_bedrooms` column. We used `SimpleImputer` to due this, which was a bit of overkill -- we knew that we were only missing values in a single column, but it was useful to see how to utilize `SimpleImputer` since it may save us time in the future. The features we engineered, including `bedrooms_per_household` and `bedrooms_per_room` list no missing values, but this is misleading -- in all of the cases where `total_bedrooms` were missing initially, these columns have 0's so we will need to re-compute them before building a model. We'll do that later.

##### Handling Text and Categoricals

Now we need to figure out how to handle the `ocean_proximity` column. Remember that this column is not a numerical column -- this poses an issue -- how do we plug `INLAND` into a mathematical equation to help us predict `median_house_value`? We must first change these values to some numerical encoding.

In [ ]:
train_set["ocean_proximity"].value_counts()

We can see from the frequency table that `ocean_proximity` is an *ordinal* categorical variable -- these are levels which can be ordered approximately by distance from the ocean: `ISLAND` followed by `NEAR BAY` followed by `NEAR OCEAN` followed by `<1H OCEAN` and finally `INLAND`. Note that the `NEAR BAY` and `NEAR OCEAN` levels are somewhat ambiguous as to their order -- I essentially flipped a coin here.

There are multiple ways to encode categorical predictors. We can use the `get_dummies()` method with the `drop_first = True` argument for *one-hot* encoding. This would create four *dummy variables*, each taking the value 0 or 1, to indicate which level the observation corresponds to -- at most one dummy variable corresponding to a given categorical variable make take the value 1 at a time. For example, we may have a dummy variable for each of `ISLAND`, `NEAR BAY`, `NEAR OCEAN`, and `INLAND` -- the dummy variable for `INLAND` may take the value 1, indicating that the row in our data frame corresponds to a neighborhood which sits inland -- it would be meaningless for both the `INLAND` and `ISLAND` dummy variables to take the value 1 simultaneously -- no neighborhood is both inland and on an island. Finally, if all of the dummy variables take on the value 0, we know that the corresponding neighborhood must belong to the missing level (sometimes called the *base level*) -- here that would be `<1H OCEAN`. 

Another option is to use `sklearn`'s `OneHotEncoder` and `OrdinalEncoder`. It is typically better to use `sklearn`'s specific tools since they are designed to work together -- using `sklearn`, we can chain transformers together, ultimately leading to a predictor (model which generates a prediction). 
$$\text{transformer} \to \text{transformer} \to \cdots \to \text{transformer} \to \text{predictor}$$ 
Such a strategy is called building a *pipeline*.

Since there is no concern with any information from the *test* or *safe* sets leaking into the *training* data with categorical variable encoding, we will just *fit* and *transform* the encoder here on the categorical column(s) of the original dataset. For simplicity, we will utilize one-hot encoding.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()

housing_cat = housing[["ocean_proximity"]]

housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

You may not have expected that -- the result of fitting and transforming the encoder on the data frame is a *sparse matrix*. There is a reason for this -- a sparse matrix is a matrix which contains very few non-zero entries -- by storing the location of the non-zero entries rather than the entire matrix itself, we conserve memory. This is particularly useful if we attempt to encode categoricals with a few thousand levels. We can convert the sparse matrix back to an array as follows.

In [ ]:
housing_cat_1hot.toarray()

We can see which level is indicated by each column of the array by asking for the `categories_` attribute of the `cat_encoder` after fitting and transforming.

In [ ]:
cat_encoder.categories_

### Recap so far...

Let's take a moment to just recap what we've done up to this point. We read in a dataset, split into training, test, and safe sets, did some exploratory analysis, filled in missing values via imputation, engineered new numerical features and saw an improvement in terms of correlation with our response variable, and converted a categorical variable into several dummy variable columns. There were a few times where we decided we wanted to alter the original dataset (when we used the imputer function and when we engineered the *per-household* features) -- in these cases, we took care to ensure that no information was leaked from the *test* or *safe* sets into the *training* data, transformed the original dataset, and then re-split into *training*, *test*, and *safe* sets -- since we set a seed for random number generation, we were ensured that the observations which were originally part of the training data remained part of the training data (etc.). 

If you've been following along closely, you've noticed that our original dataset is now fractured into two pieces -- `housing_num` (numerical columns) and `housing_cat` (categorical columns). We need to combine these back together and re-split into the *training*, *test*, and *safe* sets before we can build and validate any models.

#### Combining `housing_num` and `housing_cat`

We'll start by dropping the `bedrooms_per_household` and `bedrooms_per_room` columns since we still need to recompute those columns (we never recomputed them after imputing the `total_bedrooms` column). Since this is the case, we'll drop those columns and immediately recompute them.

In [ ]:
housing_num_tr.drop(["bedrooms_per_household", "bedrooms_per_room"], axis = 1, inplace = True)
housing_num_tr["bedrooms_per_household"] = housing_num_tr["total_bedrooms"] / housing_num_tr["households"]
housing_num_tr["bedrooms_per_room"] = housing_num_tr["total_bedrooms"] / housing_num_tr["total_rooms"]
housing_num_tr.head()

Okay, now that we've done that, let's see if we can convert `housing_cat_1hot` back to a recognizable data frame. We'll us the `pandas` `DataFrame()` method to convert the `housing_cat_1hot` object into an array. Remember that this object is a sparse matrix, so we'll need to convert it to an array first -- we can do that with the `toarray()` method as we did earlier. We'll identify that the indices for this data frame should match the indices for the original `housing` data frame, and we'll supply the list of column names matching the order of the output from `cat_encoder.categories_` which we saw earlier.

In [ ]:
housing_cat_tr = pd.DataFrame(housing_cat_1hot.toarray(), index = housing.index, columns = ["<1hOcean", "Inland", "Island", "NearBay", "NearOcean"])
housing_cat_tr.head()

Recall that with one-hot encoding we can drop a level without losing any information. We'll drop the `Inland` column here, since that sits clearly at one end of the spectrum of *proximity to ocean* locations.

In [ ]:
housing_cat_tr.drop(["Inland"], axis = 1, inplace = True)
housing_cat_tr.head()

Now that we've converted back to a data frame and dropped the base level, all while ensuring that observations have remained in the same row positions as they were originally organized into, we are ready to join the data back together.

In [ ]:
housingfinal = pd.concat([housing_num_tr, housing_cat_tr], axis = 1)
housingfinal.head()

Now that all the data is back together, we re-run our custom `split_train_test_safe()` function to obtain the *training*, *test*, and *safe* sets.

In [ ]:
train_set, test_set, safe_set = split_train_test_safe(housingfinal, train_ratio = 0.75, test_ratio = 0.15, rand_seed = 370)
train_set.head()

### Selecting and training a model

Now we are finally ready to train and validate a model. Since this notebook is already quite lengthy, we are just going to build and validate a single model. Typically we would build many competing models, validate them on the *test* data, adjust the models and retrain them, re-validate on the *test* data, iterate the adjust-retrain-revalidate sequence several times, and then finally select a best model, and validate it once more on the *safe* set. We will just go through a single round of training and validation.

Since our response variable is `median_house_value` (a numerical response), we will apply a regression technique. We'll use *linear regression* here -- a technique we will explore in much greater detail later. We first import the `LinearRegression` model class from the `slearn.linear_model` library. We then separate the predictors and response variable into separate `X_train` and `y_train` objects (since we'll need to do the same for the *test* and *safe* data eventually, we will do it here as well). Once we have these, we will fit the model on the training data.

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()

X_train = train_set.drop(["median_house_value"], axis = 1)
y_train = train_set["median_house_value"]
X_test = test_set.drop(["median_house_value"], axis = 1)
y_test = test_set["median_house_value"]
X_safe = safe_set.drop(["median_house_value"], axis = 1)
y_safe = safe_set["median_house_value"]

lin_reg.fit(X_train, y_train)

Now that we have a model built, we can use that model to make predictions for the median house values in new neighborhoods. Since we set aside the *test* and *safe* data, we have the opportunity to simulate that new, unseen data. Let's see how our model does on the first five observations in each set. We typically will measure performance on the entire sets rather than just a few observations -- we only do so here in order to illustrate what is happening when we make predictions or measure performance.

In [ ]:
print("First 5 training set predictions: ", lin_reg.predict(X_train[:5]))
print("First 5 test set predictions: ", lin_reg.predict(X_test[:5]))
print("First 5 safe set predictions: ", lin_reg.predict(X_safe[:5]))

### Model Validation

Now that we've made predictions, we would like to know how "good" (or bad) those predictions were. The entirety of Chapter 5 is dedicated to model validation and error metrics, so we'll leave much of this discussion until that point. For now, I'll ask you to believe me that a metric called *Root Mean Square Error* exists, and is similar to measuring the standard deviation of the prediction errors. We will use that metric here and, for now, I'll just tell you that other metrics exist too.

The formula for root mean square error (RMSE) is as follows:
$$\displaystyle{\text{RMSE} = \sqrt{\frac{\sum{\left(\text{predicted} - \text{observered}\right)^2}}{n}}}$$

There are some slight variations on RMSE which penalize for model complexity and amount of training data, but the denominator in the metric is all that changes. For this metric, you'll notice that we identify the **error** in each prediction, **square** each error, **average** them, and take the square **root** -- reading backwards, you'll see the rationale for the name of this metric. More to come on this error metric (and others) when we discuss Chapter 5.

In [ ]:
print("RMSE of first 5 training errors: ", ((((lin_reg.predict(X_train[:5]) - y_train[:5])**2).mean())**0.5))
print("RMSE of first 5 test errors: ", ((((lin_reg.predict(X_test[:5]) - y_test[:5])**2).mean())**0.5))
print("RMSE of first 5 safe errors: ", ((((lin_reg.predict(X_safe[:5]) - y_safe[:5])**2).mean())**0.5))

We were never just interested in the first five neighborhoods in each set -- we are interested in how our model performs "on the whole" in each of our sets. Let's check that now! In Python, notice that we write $x^n$ as `x**n`.

In [ ]:
print("Training Error (RMSE): ", ((((lin_reg.predict(X_train) - y_train)**2).mean())**0.5))
print("Test Error (RMSE): ", ((((lin_reg.predict(X_test) - y_test)**2).mean())**0.5))
print("Safe Error (RMSE): ", ((((lin_reg.predict(X_safe) - y_safe)**2).mean())**0.5))

As we've noticed before, `sklearn` has some built-in methods so that we don't need to do all of this work on our own. Below we import the `mean_squared_error` metric from the `sklearn.metrics` library and then score our model. Notice that the results are the same.

In [ ]:
from sklearn.metrics import mean_squared_error
train_preds = lin_reg.predict(X_train)
print("Training Error (RMSE): ", mean_squared_error(y_train, train_preds)**0.5)
test_preds = lin_reg.predict(X_test)
print("Test Error (RMSE): ", mean_squared_error(y_test, test_preds)**0.5)
safe_preds = lin_reg.predict(X_safe)
print("Safe Error (RMSE): ", mean_squared_error(y_safe, safe_preds)**0.5)

From the error metrics thus far, we can see that the errors made in predicting `median_house_value` on the *training*, *test*, and *safe* datasets are all quite similar. This is evidence that we are not *overfitting* our model, but these errors are quite high -- we may be *underfitting* the model. Additionally, the fact that the error metric evaluated on the unseen datasets is lower than the error metric evaluated on the training data suggests that we are likely underfitting our model. 

There is much more for us to learn! Check in on the next notebook where we will discuss error metrics and methods for predictive model validation.